In [3]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


data = pd.read_excel('C:\\Users\\calda\\Downloads\\clasificacion_siniestros.xlsx')
data.head()


,Unnamed: 0,ID_FURAT_FUREP_IGDACMLMASOLICITUDES,emp_Id_IGDACMLMASOLICITUDES,ID_ACT_ECONOMICA_IGDACMLMASOLICITUDES,seg_idPonderado_IGDACMLMASOLICITUDES,ID_SOLICITUD_IGDACMLMASOLICITUDES,FECHA_SOLICITUD_IGDACMLMASOLICITUDES,FECHA_MODIFICACION_AUD_IGDACMLMASOLICITUDES,DTO_IGDACMLMASOLICITUDES,PCL_IGDACMLMASOLICITUDES,...,ID_DX_IGACCTMIMVDIAGNOSTICOS,TIPO_CALIFICADOR_IGACCTMIMVDIAGNOSTICOS,TIPO_CALIFICACION_IGACCTMIMVDIAGNOSTICOS,FECHA_MODIFICACION_AUD_IGACCTMIMVDIAGNOSTICOS,ID_CALIFICACION_DTO_IGDACTMLMACALIFICACIONORIGEN,ID_SOLICITUD_IGDACTMLMACALIFICACIONORIGEN,ORIGEN_IGDACTMLMACALIFICACIONORIGEN,FECHA_DICTAMEN_IGDACTMLMACALIFICACIONORIGEN,FECHA_ESTRUCTURACION_IGDACTMLMACALIFICACIONORIGEN,FECHA_MODIFICACION_AUD_IGDACTMLMACALIFICACIONORIGEN
0,0,25460364,105339902,NaN,-1,1283816,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,S,S,...,NaN,NaN,NaN,NaN,1283816,1207565,1,2014-11-12 14:01:40+00:00,2014-10-22 00:00:00+00:00,2016-05-13 07:20:25+00:00
1,1,25460077,105667103,NaN,11,1283650,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,S,S,...,NaN,NaN,NaN,NaN,1283650,1205523,2,2014-11-11 16:46:02+00:00,2014-11-07 00:00:00+00:00,2014-11-11 16:46:02+00:00
2,2,25460269,105320992,NaN,6,1283758,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,S,S,...,NaN,NaN,NaN,NaN,1283758,1207289,2,2014-11-12 11:21:34+00:00,2014-11-10 00:00:00+00:00,2014-11-12 11:21:34+00:00
3,3,25460297,105734010,NaN,6,1283778,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,S,S,...,NaN,NaN,NaN,NaN,1283778,1206420,2,2014-11-12 11:47:49+00:00,2014-11-10 00:00:00+00:00,2014-11-12 11:47:49+00:00
4,4,25460248,104927847,NaN,-1,1283744,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,S,S,...,NaN,NaN,NaN,NaN,1283744,1207060,2,2014-11-12 10:47:28+00:00,2014-11-11 00:00:00+00:00,2014-11-12 10:47:28+00:00


In [5]:

preprocessed_data = data.copy()

In [6]:
# Builds a report on the percentage of data that is blank or missing in the dataset
def missing_data_report(data):
    missing_data = data.isnull().sum()
    total = data.isnull().count()
    percent = (missing_data / total) * 100
    missing_data = pd.concat([missing_data, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

display(missing_data_report(preprocessed_data).sort_values(by='Percent', ascending=False).head(20))


,Total,Percent
FECHA_AVISO_MUERTE_IGATEPMAFURAT,63160,99.993667
MONTO_RESERVA_IGATEPMAFURAT,62987,99.719777
DIAS_INCAPACIDAD_IGATEPMAFURAT,62955,99.669115
FECHA_MUERTE_IGATEPMAFURAT,62949,99.659616
ESTADO_RESERVA_IGATEPMAFURAT,61057,96.664239
MUERTE_POSTERIOR_IGATEPMAFURAT,60995,96.566082
TIPO_CALIFICADOR_IGACCTMIMVDIAGNOSTICOS,54597,86.436894
NOMBRE_DIAGNOSTICO_IGACCTMIMVDIAGNOSTICOS,54563,86.383066
ID_SOLICITUD_IGACCTMIMVDIAGNOSTICOS,54561,86.379900
CONS_DIAG_IGACCTMIMVDIAGNOSTICOS,54561,86.379900


In [7]:
# Se eliminan las columnas que tienen más del 75% de datos faltantes

preprocessed_data = preprocessed_data.dropna(thresh=preprocessed_data.shape[0]*0.25, axis=1)
preprocessed_data.head()

display(missing_data_report(preprocessed_data).sort_values(by='Percent', ascending=False).head())

# Se reemplazan los valores faltantes de las columnas restantes por el valor "DATO FALTANTE"
preprocessed_data = preprocessed_data.fillna('DATO FALTANTE')

,Total,Percent
NOMBRE_OCUPACION_IGATEPMAFURAT,11918,18.868343
OTRO_SITIO_OCURRENCIA_IGATEPMAFURAT,11908,18.852511
OTRO_TIPO_LESION_IGATEPMAFURAT,11902,18.843012
OTRO_MECANISMO_ACCIDENTE_IGATEPMAFURAT,11898,18.836679
ID_OCUPACION_AT_IGATEPMAFURAT,11671,18.477297


In [8]:

# Se convierten a timestamp las columnas que contienen 'FECHA' en su nombre
date_columns = [col for col in preprocessed_data.columns if 'FECHA' in col]
preprocessed_data[date_columns] = preprocessed_data[date_columns].apply(pd.to_datetime, errors='coerce')
preprocessed_data[date_columns].head()

# Para las columnas con menos de 5 valores únicos de texto, se convierten a tipo categórico
text_columns = preprocessed_data.select_dtypes(include='object').columns
for col in text_columns:
    if preprocessed_data[col].nunique() < 5:
        preprocessed_data[col] = preprocessed_data[col].astype('category')

# Encodes the categorical columns using one-hot encoding

preprocessed_data = pd.get_dummies(preprocessed_data, columns=preprocessed_data.select_dtypes(include='category').columns)
display(preprocessed_data.head())





,Unnamed: 0,ID_FURAT_FUREP_IGDACMLMASOLICITUDES,emp_Id_IGDACMLMASOLICITUDES,seg_idPonderado_IGDACMLMASOLICITUDES,ID_SOLICITUD_IGDACMLMASOLICITUDES,FECHA_SOLICITUD_IGDACMLMASOLICITUDES,FECHA_MODIFICACION_AUD_IGDACMLMASOLICITUDES,TIPO_SINIESTRO_IGDACMLMASOLICITUDES,FECHA_SINIESTRO_IGDACMLMASOLICITUDES,ID_FURAT_IGATEPMAFURAT,...,ID_TIPO_DOC_EMP_IGDACMLMASOLICITUDES_NI,IND_MUERTE_IGATEPMAFURAT_0.0,IND_MUERTE_IGATEPMAFURAT_1.0,IND_MUERTE_IGATEPMAFURAT_DATO FALTANTE,ACCIDENTE_GRAVE_IGATEPMAFURAT_0,ACCIDENTE_GRAVE_IGATEPMAFURAT_N,ACCIDENTE_GRAVE_IGATEPMAFURAT_S,RIESGO_BIOLOGICO_IGATEPMAFURAT_0,RIESGO_BIOLOGICO_IGATEPMAFURAT_N,RIESGO_BIOLOGICO_IGATEPMAFURAT_S
0,0,25460364,105339902,-1,1283816,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-05 00:00:00+00:00,25460364,...,True,True,False,False,False,True,False,False,True,False
1,1,25460077,105667103,11,1283650,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-04 00:00:00+00:00,25460077,...,False,True,False,False,False,True,False,False,True,False
2,2,25460269,105320992,6,1283758,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-05 00:00:00+00:00,25460269,...,True,True,False,False,False,True,False,False,True,False
3,3,25460297,105734010,6,1283778,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-04 00:00:00+00:00,25460297,...,True,True,False,False,False,True,False,False,True,False
4,4,25460248,104927847,-1,1283744,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-05 00:00:00+00:00,25460248,...,False,True,False,False,False,True,False,False,True,False


In [9]:
!python -m spacy download es_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 320.0 kB/s eta 0:00:41
      --------------------------------------- 0.3/12.9 MB 3.2 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/12.9 MB 7.7 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.9 MB 14.4 MB/s eta 0:00:01
     ------------ --------------------------- 3.9/12.9 MB 17.7 MB/s eta 0:00:01
     ----------------- ---------------------- 5.8/12.9 MB 21.7 MB/s eta 0:00:01
     ----------------------- ---------------- 7.6/12.9 MB 24.2 MB/s eta 0:00:01
     ----------------------------- ---------- 9.5/12.9 MB 26.2 MB/s eta 0:00:01
     ---------------------------------- ---- 11.4/12.9 MB 36.3 MB/s eta 0:00:01
     --------------------------------------  12.9/12.9 MB 38.5 MB/s eta 0:00:01
     --------------------------------------  12.9/12

In [10]:
nlp = spacy.load("es_core_news_sm")
df = preprocessed_data.copy()
text_columns = ["NOMBRE_OCUPACION_IGATEPMAFURAT", "DESCRIPCION_AT_IGATEPMAFURAT"]

def clean_text(text):
    """
    Limpia el texto eliminando los números, la puntuación y convirtiendo el texto a minúsculas
    """
    if pd.isna(str(text)):
        return "N/A"
    text = re.sub(r'\d+', '', str(text))  
    text = re.sub(r'[^\w\s]', '', str(text))  
    text = text.lower()  
    return text

for col in text_columns:
    print(f"Cleaning text in column: {col}")
    df[f"clean_{col}"] = df[col].apply(clean_text)

def process_text(text):
    """
    Tokeniza el texto, elimina las palabras vacías y la puntuación, y lematiza las palabras
    """
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

for col in text_columns:
    df[f"processed_{col}"] = df[f"clean_{col}"].apply(process_text)

vectorizers = {}
tfidf_dfs = []

for col in text_columns:
    vectorizer = TfidfVectorizer(max_features=100)  
    tfidf_matrix = vectorizer.fit_transform(df[f"processed_{col}"])
    
    vectorizers[col] = vectorizer
    
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f"{col}_{word}" for word in vectorizer.get_feature_names_out()])
    tfidf_dfs.append(tfidf_df)

for col in text_columns:
    df[f"{col}_text_length"] = df[f"processed_{col}"].apply(lambda x: len(x.split()))
    df[f"{col}_num_unique_words"] = df[f"processed_{col}"].apply(lambda x: len(set(x.split())))

final_df = pd.concat([df] + tfidf_dfs, axis=1)

final_df.head()

Cleaning text in column: NOMBRE_OCUPACION_IGATEPMAFURAT
Cleaning text in column: DESCRIPCION_AT_IGATEPMAFURAT


,Unnamed: 0,ID_FURAT_FUREP_IGDACMLMASOLICITUDES,emp_Id_IGDACMLMASOLICITUDES,seg_idPonderado_IGDACMLMASOLICITUDES,ID_SOLICITUD_IGDACMLMASOLICITUDES,FECHA_SOLICITUD_IGDACMLMASOLICITUDES,FECHA_MODIFICACION_AUD_IGDACMLMASOLICITUDES,TIPO_SINIESTRO_IGDACMLMASOLICITUDES,FECHA_SINIESTRO_IGDACMLMASOLICITUDES,ID_FURAT_IGATEPMAFURAT,...,DESCRIPCION_AT_IGATEPMAFURAT_seãor,DESCRIPCION_AT_IGATEPMAFURAT_sintio,DESCRIPCION_AT_IGATEPMAFURAT_subir,DESCRIPCION_AT_IGATEPMAFURAT_tobillo,DESCRIPCION_AT_IGATEPMAFURAT_trabajador,DESCRIPCION_AT_IGATEPMAFURAT_trabajadora,DESCRIPCION_AT_IGATEPMAFURAT_trabajo,DESCRIPCION_AT_IGATEPMAFURAT_tubo,DESCRIPCION_AT_IGATEPMAFURAT_vehiculo,DESCRIPCION_AT_IGATEPMAFURAT_él
0,0,25460364,105339902,-1,1283816,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-05 00:00:00+00:00,25460364,...,0.0,0.0,0.0,0.0,0.160821,0.0,0.000000,0.0,0.0,0.000000
1,1,25460077,105667103,11,1283650,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-04 00:00:00+00:00,25460077,...,0.0,0.0,0.0,0.0,0.345690,0.0,0.000000,0.0,0.0,0.000000
2,2,25460269,105320992,6,1283758,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-05 00:00:00+00:00,25460269,...,0.0,0.0,0.0,0.0,0.156254,0.0,0.398704,0.0,0.0,0.371796
3,3,25460297,105734010,6,1283778,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-04 00:00:00+00:00,25460297,...,0.0,0.0,0.0,0.0,0.110602,0.0,0.000000,0.0,0.0,0.131585
4,4,25460248,104927847,-1,1283744,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0,2009-03-05 00:00:00+00:00,25460248,...,0.0,0.0,0.0,0.0,0.199085,0.0,0.000000,0.0,0.0,0.236854


In [11]:
def log_transform(data, columns):
    """
    Aplica una transformación logarítmica a las columnas que tienen una kurtosis mayor a 5 o un sesgo mayor a 1
    """
    for col in columns:
        if data[col].kurtosis() > 5 or data[col].skew() > 1:
            data[col] = np.log1p(data[col])
    return data

numeric_columns = final_df.select_dtypes(include='number').columns
final_df = log_transform(final_df, numeric_columns)

def z_score_outliers(data, columns):
    """
    Detecta y elimina los valores atípicos en las columnas numéricas utilizando la puntuación Z
    """
    for col in columns:
        data[col] = data[col][(data[col] - data[col].mean()).abs() < 3 * data[col].std()]
    return data

final_df = z_score_outliers(final_df, numeric_columns)


c:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
scaler = MinMaxScaler()
final_df[numeric_columns] = scaler.fit_transform(final_df[numeric_columns])

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [13]:
display(final_df)

,Unnamed: 0,ID_FURAT_FUREP_IGDACMLMASOLICITUDES,emp_Id_IGDACMLMASOLICITUDES,seg_idPonderado_IGDACMLMASOLICITUDES,ID_SOLICITUD_IGDACMLMASOLICITUDES,FECHA_SOLICITUD_IGDACMLMASOLICITUDES,FECHA_MODIFICACION_AUD_IGDACMLMASOLICITUDES,TIPO_SINIESTRO_IGDACMLMASOLICITUDES,FECHA_SINIESTRO_IGDACMLMASOLICITUDES,ID_FURAT_IGATEPMAFURAT,...,DESCRIPCION_AT_IGATEPMAFURAT_seãor,DESCRIPCION_AT_IGATEPMAFURAT_sintio,DESCRIPCION_AT_IGATEPMAFURAT_subir,DESCRIPCION_AT_IGATEPMAFURAT_tobillo,DESCRIPCION_AT_IGATEPMAFURAT_trabajador,DESCRIPCION_AT_IGATEPMAFURAT_trabajadora,DESCRIPCION_AT_IGATEPMAFURAT_trabajo,DESCRIPCION_AT_IGATEPMAFURAT_tubo,DESCRIPCION_AT_IGATEPMAFURAT_vehiculo,DESCRIPCION_AT_IGATEPMAFURAT_él
0,0.000000,0.531789,0.624331,NaN,0.696580,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0.0,2009-03-05 00:00:00+00:00,0.531789,...,0.0,0.0,0.0,0.0,0.382338,0.0,0.0,0.0,0.0,0.000000
1,0.000016,0.531778,0.843479,NaN,0.696490,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0.0,2009-03-04 00:00:00+00:00,0.531778,...,0.0,0.0,0.0,0.0,0.761220,0.0,0.0,0.0,0.0,0.000000
2,0.000032,0.531785,0.611645,NaN,0.696549,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0.0,2009-03-05 00:00:00+00:00,0.531785,...,0.0,0.0,0.0,0.0,0.372232,0.0,NaN,0.0,0.0,0.713716
3,0.000047,0.531787,0.888207,NaN,0.696560,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0.0,2009-03-04 00:00:00+00:00,0.531787,...,0.0,0.0,0.0,0.0,0.268952,0.0,0.0,0.0,0.0,0.279098
4,0.000063,0.531785,0.347380,NaN,0.696541,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0.0,2009-03-05 00:00:00+00:00,0.531785,...,0.0,0.0,0.0,0.0,0.465487,0.0,0.0,0.0,0.0,0.479929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63159,0.999937,0.999564,0.568263,NaN,0.029425,2009-08-20 15:49:23+00:00,2009-09-01 11:30:22+00:00,0.5,2009-08-19 00:00:00+00:00,0.999564,...,0.0,0.0,0.0,0.0,0.324611,0.0,0.0,0.0,0.0,0.336226
63160,0.999953,0.999562,0.133013,NaN,0.029384,2009-08-20 15:08:09+00:00,2009-08-20 15:08:09+00:00,0.5,2009-08-20 00:00:00+00:00,0.999562,...,0.0,0.0,0.0,0.0,0.370923,0.0,0.0,0.0,0.0,0.383606
63161,0.999968,0.531934,0.648610,NaN,0.697404,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0.0,2009-03-12 00:00:00+00:00,0.531934,...,0.0,0.0,0.0,0.0,0.241844,0.0,0.0,0.0,0.0,0.251200
63162,0.999984,0.531925,0.549869,NaN,0.697335,2015-02-06 00:00:00+00:00,2015-02-06 00:00:00+00:00,0.0,2009-03-10 00:00:00+00:00,0.531925,...,0.0,0.0,0.0,0.0,0.375954,0.0,0.0,NaN,0.0,0.000000


In [14]:
final_df = final_df.drop(columns=text_columns + [f"clean_{col}" for col in text_columns] + [f"processed_{col}" for col in text_columns])
final_df = final_df.drop(columns='Unnamed: 0')

In [15]:
final_df.to_csv('clasi_siniestros_preprocesado.csv', index=False)